# Imports and Preprocessing

In [7]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_Multi_Raw')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

target_test = target_test['product_1_demand']
target_train = target_train['product_1_demand']

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_Multi_Raw


In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing for numeric columns (scale them)
numeric_features = ['temperature']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = ['location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

# Preprocessing on train data
X_train = preprocessor.fit_transform(data_train)

# Preprocessing on test data
X_test = preprocessor.transform(data_test)


print(X_test)

[[-1.2614810567465224 0.0 0.0 ... 5 2022 False]
 [0.03038460101498815 0.0 0.0 ... 9 2020 False]
 [-0.2866376462884491 0.0 0.0 ... 5 2020 False]
 ...
 [-1.2614810567465224 0.0 0.0 ... 1 2021 False]
 [-1.0316399274515304 0.0 0.0 ... 1 2022 False]
 [0.03038460101498815 0.0 0.0 ... 4 2020 True]]


# Demand Forecasting with ANN Model

In [9]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal
import numpy as np

# Define function to create the ANN model 
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[15]): 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation) 

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_ANN.fit(X_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Predict the target values
target_pred_ANN = rnd_search_cv_ANN.predict(X_test)
train_pred_ANN = rnd_search_cv_ANN.predict(X_train)

mse = mean_squared_error(target_test, target_pred_ANN)
print("MSE:", mse)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 16, 'epochs': 30, 'learning_rate': 0.0002477669356103843, 'n_hidden': 0, 'n_neurons': 32}
MSE: 25.876046273604732


# Demand Forecasting with DT (LightGBM)

In [10]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_DT.fit(X_train, target_train)

# Get the best parameters and best estimator
best_params = rnd_search_cv_DT.best_params_
best_estimator = rnd_search_cv_DT.best_estimator_

# Make predictions
target_pred_DT = best_estimator.predict(X_test)
train_pred_DT = best_estimator.predict(X_train)

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_DT)
print("MSE:", mse)



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 80
[LightGBM] [Info] Number of data points in the train set: 533, number of used features: 14
[LightGBM] [Info] Start training from score 49.352720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

# Solving Newsvendor (Non-parametric)

In [11]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((c - s) + (p - c))

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten
forecast_error_ANN_std = np.std(forecast_error_ANN)

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=scenario_size) 

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = norm.ppf(critical_ratio,loc=demand,scale=forecast_error_ANN_std)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_ANN.append(final_allocation)

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_DT = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten
forecast_error_DT_std = np.std(forecast_error_DT)

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=scenario_size) 

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = norm.ppf(critical_ratio,loc=demand,scale=forecast_error_DT_std)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT.append(final_allocation)



# Solving Newsvendor (model-based)

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_ANN_flatten = train_pred_ANN.flatten()
forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten
forecast_error_ANN_std = np.std(forecast_error_ANN) 

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_ANN[i] + np.random.normal(loc=0, scale=forecast_error_ANN_std, size=scenario_size)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = norm.ppf(critical_ratio,loc=demand,scale=forecast_error_ANN_std)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_ANN_model.append(final_allocation)

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100

# Initialize an empty list to store the final order quantities
final_order_quantities_DT_model = []

# Calculate the forecast error for ANN on the prediction of the training data
target_train_flatten = target_train.values.flatten()
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train_flatten - train_pred_DT_flatten
forecast_error_DT_std = np.std(forecast_error_DT)

# Loop over each week in data_test
for i in range(len(data_test)):

    # Create Demand Scenarios for this week
    demand_scenarios = target_pred_DT[i] + np.random.normal(loc=0, scale=forecast_error_DT_std, size=scenario_size)

    # Initialize a list to store the solutions for each scenario
    saa_solutions = []

    # For each demand scenario, solve the newsvendor problem
    for demand in demand_scenarios:
        
        # Calculate the solution for this scenario
        solution = norm.ppf(critical_ratio,loc=demand,scale=forecast_error_DT_std)

        # Store the solution for this scenario
        saa_solutions.append(solution)

    # Average the solutions to get the final allocation
    final_allocation = np.mean(saa_solutions, axis=0)

    # Store the final order quantities
    final_order_quantities_DT_model.append(final_allocation)

# Evaluation

In [16]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0
overall_costs_ANN_model = 0
overall_costs_DT_model = 0

for i in range(len(target_test)):

    # Calculate understock and overstock costs
    cost_ANN = 0
    cost_DT = 0
    cost_ANN_model = 0
    cost_DT_model = 0

    # Costs for non-parametric
    if final_order_quantities_ANN[i] < target_test.values[i]:
        cost_ANN = (p - c) * (target_test.values[i] - np.round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        cost_ANN = (c - s) * (np.round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        cost_DT = (p - c) * (target_test.values[i] - np.round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        cost_DT = (c - s) * (np.round(final_order_quantities_DT[i]) - target_test.values[i])
    # Costs for model-based
    if final_order_quantities_ANN_model[i] < target_test.values[i]:
        cost_ANN_model = (p - c) * (target_test.values[i] - np.round(final_order_quantities_ANN_model[i]))
    if final_order_quantities_ANN_model[i] > target_test.values[i]:
        cost_ANN_model = (c - s) * (np.round(final_order_quantities_ANN_model[i]) - target_test.values[i])
    if final_order_quantities_DT_model[i] < target_test.values[i]:
        cost_DT_model = (p - c) * (target_test.values[i] - np.round(final_order_quantities_DT_model[i]))
    if final_order_quantities_DT_model[i] > target_test.values[i]:
        cost_DT_model = (c - s) * (np.round(final_order_quantities_DT_model[i]) - target_test.values[i])

    # Calculate the total costs for the week
    overall_costs_ANN += cost_ANN
    overall_costs_DT += cost_DT
    overall_costs_ANN_model += cost_ANN_model
    overall_costs_DT_model += cost_DT_model
   

# Print the overall profit
print('Overall costs for ANN (non-parametric): ', int(overall_costs_ANN))
print('Overall costs for DT (non-parametric): ', int(overall_costs_DT))
print('Overall costs for ANN (model-based): ', int(overall_costs_ANN_model))
print('Overall costs for DT (model-based): ', int(overall_costs_DT_model))

Overall costs for ANN (non-parametric):  760
Overall costs for DT (non-parametric):  753
Overall costs for ANN (model-based):  743
Overall costs for DT (model-based):  173
